### EDA of Apr 2021 kaggle TPS

This notebook is used to do initial exploration of the training data, get an understanding of the distribution and completeness of each column and relationship with the target. The aim is to build a broad understanding and make some baseline predictions.

Synthanic [Data Dictionary](https://www.kaggle.com/c/tabular-playground-series-apr-2021/data?select=train.csv)

In [ ]:
# imports for EDA
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# imports for inference
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [ ]:
df = pd.read_csv("../data/raw/train.csv")

In [ ]:
df.head()

We can get some idea of what to expect from the first few rows - next to look at the distribution of each more closely, and think about how to handle missing data and how to encode sex, class, port of embarkation etc.

In [ ]:
df.info()

In [ ]:
df.describe()

So with a 43% survival rate, hopefully we can do better than predicting everyone as lost at 57%. Noticing a lot of missing ages, not so many missing fares. 

For the purposes of this challend I'm assuming the passenger IDs are all ok - on to survival, and lots of plots.

In [ ]:
sns.set_style("whitegrid")
sns.displot(data=df, x="Survived")

In [ ]:
sns.displot(data=df, x="Pclass", hue="Survived", multiple="stack")

In [ ]:
df.groupby(['Sex', 'Survived'])['Survived'].count()

In [ ]:
df['Sex'].isnull().sum()

No missing values here - but note significant differences in outcomes between male and female.

In [ ]:
sns.displot(data=df, x="Age", hue="Survived", kind="kde")

In [ ]:
df['Age'].isnull().sum()

3.3% of values for Age are missing - we may be able to impute this from other columns

In [ ]:
sns.displot(data=df, x="SibSp", hue="Survived", multiple="stack")
# SibSp and Parch will be returned to after a baseline established. apologies for an ugly chart

In [ ]:
sns.displot(data=df, x="Parch", hue="Survived", multiple="stack")

In [ ]:
print(df['Ticket'].nunique())
print(df['Ticket'].isnull().sum())
# After baseline established, should be possible to link families using ticket, cabin

In [ ]:
print(df['Fare'].isnull().sum())
sns.displot(data=df, x="Fare", hue="Survived", kind="kde")

Those paying < 100 did not do so well. Let's take a closer look at passengers who paid more:

In [ ]:
dplot = sns.displot(data=df, x="Fare", hue="Survived", kind="kde")
dplot.set(xlim=(100, 800))
dplot.set(ylim=(0, 0.001))

In [ ]:
print(df['Cabin'].nunique())
print(df['Cabin'].isnull().sum())

In [ ]:
df.groupby(['Embarked', 'Survived'])['Survived'].count()

In [ ]:
df['Embarked'].isnull().sum()
# A few missing here, to investigate after the first iteration is complete

In [ ]:
# here's a quick/dirty remap to plot embarkation port
df['Emb'] = df['Embarked'].map({'C': 0, 'Q': 1, 'S': 2})
sns.displot(data=df, x="Emb", hue="Survived", multiple="stack")

There is work to be done to work out the best way to impute missing data around Age, SibSp, Parch, Ticket, Cabin, and Embarked, but first I want to compare a quick mvp to a dummy benchmark to set a baseline. For this the categorical columns will need to be encoded, nulls filled with temporary junk, after breaking out to a train/test split.

In [ ]:
target = df[['Survived']]
features = df[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

In [ ]:
categorical = ['Pclass', 'Sex', 'Embarked']
numeric = ['Age', 'SibSp', 'Parch', 'Fare']

In [ ]:
X_train[numeric].describe()

In [ ]:
X_train['Age'] = X_train.groupby(['Sex','Pclass'], sort=False)['Age'].apply(lambda x: x.fillna(x.median()))

In [ ]:
X_train['Fare'] = X_train.groupby(['Sex','Pclass'], sort=False)['Fare'].apply(lambda x: x.fillna(x.median()))

With numerical blanks filled with median values taken from grouping of sex and class, and a train test split specified, numeric columns are scaled and categorical columns are one hot encoded. Initial estimator is logistic regression for a simple binary classification task.

In [ ]:
preprocessor = make_column_transformer(
    (StandardScaler(), numeric),
    (OneHotEncoder(drop='if_binary'), categorical), 
    remainder='passthrough')

model = make_pipeline(
    preprocessor,
    LogisticRegression())

_ = model.fit(X_train, np.ravel(y_train))

To set a baseline for model performance, let's see a classification report if the ship is predicted as going down with no survivors.

In [ ]:
y_pred = model.predict(X_train)
y_dummy = np.full(shape=len(y_pred), fill_value=0)
print(classification_report(y_train, y_dummy))

In [ ]:
print(classification_report(y_train, y_pred))

Performance on training data is ok at 76% overall accuracy; better than guessing that no one makes it at least, at 57% accuracy. Compare to the held out test set:

In [ ]:
X_test['Age'] = X_test.groupby(['Sex','Pclass'], sort=False)['Age'].apply(lambda x: x.fillna(x.median()))
X_test['Fare'] = X_test.groupby(['Sex','Pclass'], sort=False)['Fare'].apply(lambda x: x.fillna(x.median()))
y_unseen = model.predict(X_test)
print(classification_report(y_test, y_unseen))

To finish with the same steps are carried out on the competition test set and submitted. With more time you could do EDA on the test set to look for drift across each category, but for right now I'm interested in how this model performs without any additional intervention.

This initial submission achieved a leaderboard score of 0.77000. 

Next steps are:
- using an ensemble algorithm on the same data as logistic regression above, measure uplift
- look at more intelligent ways of filling nulls, making better use of tickets and cabins to see if that affects survival.

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
# yes there needs to be a proper CV grid search eventually - submitting this as is to kaggle yielded leaderboard score of 0.78642
model_gb = make_pipeline(
    preprocessor,
    GradientBoostingClassifier())

_ = model_gb.fit(X_train, np.ravel(y_train))

In [ ]:
y_pred_gb = model_gb.predict(X_train)
print(classification_report(y_train, y_pred_gb))

In [ ]:
y_unseen_gb = model_gb.predict(X_test)
print(classification_report(y_test, y_unseen_gb))

The ensemble method achieved a leaderboard score of 0.78642 out of the box.